<a href="https://colab.research.google.com/github/saintblue97/DeepNN_Optiver/blob/main/optiver_aeguiarte_tseries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive; drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install --upgrade --force-reinstall --no-deps kaggle > log  # upgrade kaggle package (to avoid a warning)
!mkdir -p ~/.kaggle                                           # .kaggle folder must contain kaggle.json for kaggle executable to properly authenticate you to Kaggle.com
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json >log  # First, download kaggle.json from kaggle.com (in Account page) and place it in the root of mounted Google Drive
#!cp kaggle.json ~/.kaggle/kaggle.json > log                   # Alternative location of kaggle.json (without a connection to Google Drive)
!chmod 600 ~/.kaggle/kaggle.json                              # give only the owner full read/write access to kaggle.json
!kaggle config set -n competition -v optiver-trading-at-the-close        # set the competition context for the next few kaggle API calls. !kaggle config view - shows current settings
!kaggle competitions download >> log                          # download competition dataset as a zip file
!unzip -o *.zip >> log                                        # Kaggle dataset is copied as a single file and needs to be unzipped.
# !kaggle competitions leaderboard --show                       # print public leaderboard

- competition is now set to: optiver-trading-at-the-close


In [3]:
%%time
%%capture
%reset -f
import numpy as np, pandas as pd, time, matplotlib.pyplot as plt, seaborn as sns, os, tqdm, re, sys, cv2, skimage, xgboost, lightgbm as lgb, librosa

ToCSV = lambda df, fname: df.round(2).to_csv(f'{fname}.csv', index_label='id') # rounds values to 2 decimals

class Timer():
  def __init__(self, lim:'RunTimeLimit'=300): self.t0, self.lim, _ = time.time(), lim, print(f'⏳ started. You have {lim} sec. Good luck!')
  def ShowTime(self):
    msg = f'Runtime is {time.time()-self.t0:.0f} sec'
    print(f'\033[91m\033[1m' + msg + f' > {self.lim} sec limit!!!\033[0m' if (time.time()-self.t0-1) > self.lim else msg)

np.set_printoptions(linewidth=100, precision=2, edgeitems=2, suppress=True)
pd.set_option('display.max_columns', 20, 'display.precision', 2, 'display.max_rows', 4)

CPU times: user 2.31 s, sys: 269 ms, total: 2.58 s
Wall time: 4 s


In [4]:
def is_google_colab():
    """Check if the environment is Google Colab.

    Returns:
        bool: True if in Google Colab, False otherwise.
    """
    try:
        import google.colab
        return True
    except ImportError:
      return False

if is_google_colab():
  file_path = ''
else:
  file_path = '/kaggle/input/optiver-trading-at-the-close/'

In [5]:
sample_submission_path = file_path + 'example_test_files/sample_submission.csv'

sample_submission = pd.read_csv(sample_submission_path); sample_submission

,time_id,row_id,target
0,26290,478_0_0,1
1,26290,478_0_1,1
...,...,...,...
32998,26454,480_540_198,1
32999,26454,480_540_199,1


## Imports

In [6]:
pd.set_option('display.max_rows', 50)

In [7]:
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, MinMaxScaler, RobustScaler
from sklearn.model_selection import cross_val_score, cross_val_predict, KFold, train_test_split, GridSearchCV
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin, clone
from sklearn.metrics import mean_absolute_error

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ARDRegression, BayesianRidge, LinearRegression, Ridge, Lars
from tqdm import tqdm

In [8]:
from operator import itemgetter

In [9]:
import tensorflow as tf
import tensorflow.keras as keras
from keras.layers import Flatten, Dense

In [10]:
SAMPLE_SIZE = 500 # Number of examples = SAMPLE_SIZE * 55
RANDOM_STATE = 40

# Load Data

In [11]:
#tmr = Timer()

In [12]:
train_data_path = file_path + 'train.csv'
df = pd.read_csv(train_data_path);
df = df.dropna(subset=['target']); df.head()

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
0,0,0,0,3.18e+06,1,1.0,1.34e+07,NaN,NaN,1.0,60651.50,1.0,8493.03,1.0,-3.03,0,0_0_0
1,1,0,0,1.67e+05,-1,1.0,1.64e+06,NaN,NaN,1.0,3233.04,1.0,20605.09,1.0,-5.52,0,0_0_1
2,2,0,0,3.03e+05,-1,1.0,1.82e+06,NaN,NaN,1.0,37956.00,1.0,18995.00,1.0,-8.39,0,0_0_2
3,3,0,0,1.19e+07,-1,1.0,1.84e+07,NaN,NaN,1.0,2324.90,1.0,479032.40,1.0,-4.01,0,0_0_3
4,4,0,0,4.48e+05,-1,1.0,1.79e+07,NaN,NaN,1.0,16485.54,1.0,434.10,1.0,-7.35,0,0_0_4


In [13]:
test_data_path = file_path + 'example_test_files/test.csv'
df_test  = pd.read_csv(test_data_path); df_test.head()

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,time_id,row_id
0,0,478,0,3.75e+06,-1,1.0,1.15e+07,NaN,NaN,1.0,22940.00,1.0,9177.60,1.0,26290,478_0_0
1,1,478,0,9.86e+05,-1,1.0,3.85e+06,NaN,NaN,1.0,1967.90,1.0,19692.00,1.0,26290,478_0_1
2,2,478,0,5.99e+05,1,1.0,4.36e+06,NaN,NaN,1.0,4488.22,1.0,34955.12,1.0,26290,478_0_2
3,3,478,0,2.87e+06,-1,1.0,2.71e+07,NaN,NaN,1.0,16082.04,1.0,10314.00,1.0,26290,478_0_3
4,4,478,0,7.40e+05,-1,1.0,8.88e+06,NaN,NaN,1.0,19012.35,1.0,7245.60,1.0,26290,478_0_4


In [14]:
print('Shape of training data =', df.shape)
print('Shape of testing data  =', df_test.shape)

Shape of training data = (5237892, 17)
Shape of testing data  = (33000, 16)


In [15]:
# Remove NaNs here!!
df_pp = df.dropna()

In [16]:
df_pp.shape

(2343638, 17)

In [17]:
def get_grouped_df(original_df:pd.DataFrame):
    grouped_df_lst = []
    incomplete_ts = []
    cnt = 0
    for k, subdf in tqdm(df.groupby(['stock_id', 'date_id'])):
        if subdf.shape[0] >= 55:
            grouped_df_lst.append(subdf)
        else:
            incomplete_ts.append({"stock_id": k[0], "date_id": k[1], "length": len(subdf)})
            #display(subdf.head(20))
            # print(subdf.shape)
    return grouped_df_lst, incomplete_ts

In [18]:
grp_df_lst, incomplete_ts_lst = get_grouped_df(df_pp)

100%|██████████| 95235/95235 [00:06<00:00, 15742.94it/s]


In [19]:
len(grp_df_lst)

95232

In [20]:
grp_df_lst[0].head()

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
0,0,0,0,3.18e+06,1,1.0,1.34e+07,NaN,NaN,1.0,60651.50,1.0,8493.03,1.0,-3.03,0,0_0_0
191,0,0,10,1.30e+06,1,1.0,1.53e+07,NaN,NaN,1.0,13996.50,1.0,23519.16,1.0,0.39,1,0_10_0
382,0,0,20,1.30e+06,1,1.0,1.53e+07,NaN,NaN,1.0,4665.50,1.0,12131.60,1.0,4.22,2,0_20_0
573,0,0,30,1.30e+06,1,1.0,1.53e+07,NaN,NaN,1.0,55998.00,1.0,46203.30,1.0,5.45,3,0_30_0
764,0,0,40,1.22e+06,1,1.0,1.53e+07,NaN,NaN,1.0,14655.95,1.0,26610.45,1.0,3.17,4,0_40_0


In [21]:
dataset_folds = KFold(n_splits=10, shuffle=True, random_state=RANDOM_STATE)
for fold_num, (train_index, test_index) in enumerate(dataset_folds.split(grp_df_lst)):
    XY_train_df_lst = list(itemgetter(*list(train_index))(grp_df_lst))[:SAMPLE_SIZE]
    XY_test_df_lst = list(itemgetter(*list(test_index))(grp_df_lst))[:SAMPLE_SIZE//10]
    break # Get one fold


In [22]:
len(XY_train_df_lst)

500

In [23]:
# Release Memory
if 'grp_df_lst' in vars(): del grp_df_lst
if 'incomplete_ts_lst' in vars(): del incomplete_ts_lst
if 'dataset_folds' in vars(): del dataset_folds

In [24]:
class DropColumns(BaseEstimator, TransformerMixin):
    def __init__(self, columns_to_drop):
        self.columns_to_drop = columns_to_drop

    def fit(self, X, y=None):
        X_orig = X
        self.n_features_in_ = X.shape[1]
        if hasattr(X_orig, "columns"):
            self.feature_names_in_ = np.array(X_orig.columns, dtype=object)
        return self

    def transform(self, X):
        # Only drop columns that exist in X
        columns_to_drop = [col for col in self.columns_to_drop if col in X.columns]
        return X.drop(columns_to_drop, axis=1)

    def get_feature_names_out(self, names=None):
        return ['hola']

    def get_feature_names_out(self, input_features=None):
        if input_features is None:
            return getattr(self, "feature_names_in_",
                           [f"x{i}" for i in range(self.n_features_in_)])
        else:
            if len(input_features) != self.n_features_in_:
                raise ValueError("Invalid number of features")
            if hasattr(self, "feature_names_in_") and not np.all(
                self.feature_names_in_ == input_features
            ):
                raise ValueError("input_features ≠ feature_names_in_")
            return input_features

class AddRatios(BaseEstimator, TransformerMixin):
    def __init__(self, col_pair_lst:list):
        self.col_pair_lst = col_pair_lst

    def fit(self, X, y=None):
        X_orig = X
        self.n_features_in_ = X.shape[1]
        if hasattr(X_orig, "columns"):
            self.feature_names_in_ = np.array(X_orig.columns, dtype=object)
        return self

    def transform(self, X):
        X_ = X.copy()
        for col1, col2 in self.col_pair_lst:
            non_zero_mask = X_[col2] != 0
            X_[f'{col1}_to_{col2}_ratio'] = np.where(non_zero_mask, X_[col1] / X_[col2], 0)
        return X_

    def get_feature_names_out(self, input_features=None):
        if input_features is None:
            return getattr(self, "feature_names_in_",
                           [f"x{i}" for i in range(self.n_features_in_)])
        else:
            if len(input_features) != self.n_features_in_:
                raise ValueError(f"Invalid number of features.\n input_features: {input_features}\n self.n_features_in_:{self.n_features_in_}")
            if hasattr(self, "feature_names_in_") and not np.all(
                self.feature_names_in_ == input_features
            ):
                raise ValueError("input_features ≠ feature_names_in_")
            return input_features + [f'{col1}_to_{col2}_ratio' for col1, col2 in self.col_pair_lst]

In [25]:
# Set up preprocessing pipeline
ratios_cols = [
      ('reference_price', 'wap'),
      ('bid_price', 'wap'),
      ('ask_price', 'wap'),
      ]
drop_cols = [
    'near_price','far_price','time_id',
    'bid_price', 'ask_price', 'wap',
    'row_id',
    ]

ds_transformer = ColumnTransformer([
        ('select_cols', 'drop', drop_cols),
        ('imputer', SimpleImputer(strategy='constant', fill_value=0), make_column_selector(dtype_include=np.number)),
        ],
    )
pp_pipeline = Pipeline([
    ('add_ratios', AddRatios(ratios_cols)),
    #('ds_transformer', ds_transformer),
    ('drop_cols', DropColumns(drop_cols)),
    ('imputer', SimpleImputer(strategy='constant', fill_value=0)),
    # ('imputer', SimpleImputer(strategy='median')),
    # ('poly', PolynomialFeatures(degree=2, include_bias=False)),
    ('scaler', StandardScaler()),
])

In [26]:
len(XY_train_df_lst)

500

In [27]:
tf.float32

tf.float32

In [28]:
sequence_length = 55
batch_size = 512

XY_dataset = None
for XY_df in tqdm(XY_train_df_lst):
    # Preprocess XY DataFrame
    Xtrain_array = pp_pipeline.fit_transform(XY_df.drop('target', axis=1))
    # Create TF Dataset
    XYtrain_ds = tf.keras.utils.timeseries_dataset_from_array(
        tf.cast(Xtrain_array, tf.float32),
        XY_df.target,
        sequence_length,
        sequence_stride=1,
        sampling_rate=1,
        batch_size=128,
        shuffle=False,
        seed=RANDOM_STATE,
        start_index=None,
        end_index=None
    )
    if XY_dataset is None:
        XY_dataset = XYtrain_ds
    else:
        XY_dataset = XY_dataset.concatenate(XYtrain_ds)
    #break

100%|██████████| 500/500 [00:32<00:00, 15.30it/s]


In [29]:
NN_input_shape = Xtrain_array.shape[1] ; NN_input_shape

12

In [30]:
#len([None for _ in XY_dataset])
#

# Make submission

In [31]:
# from sklearn.metrics import make_scorer,mean_absolute_error
# from sklearn.linear_model import LogisticRegression, LogisticRegressionCV, PassiveAggressiveClassifier, Perceptron, RidgeClassifier, RidgeClassifierCV, SGDClassifier, SGDOneClassSVM
# from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis

# if is_google_colab():

#     models = [
#         LinearRegression(),
#         # Ridge(),
#         # BayesianRidge(),
#         # LogisticRegression(max_iter=5000),
#     ]


#     for model in models:
#         model_pipeline = Pipeline(preprocess_steps + [('model', model)])

#         scores = cross_val_score(model_pipeline, tX.copy(), tY.copy(), cv=5, scoring='neg_mean_absolute_error')

#         print(f'model={model} | score={np.mean(scores):.3f}{chr(177)} {np.std(scores):.3f}\n')

In [32]:
def DNN(input_shape:int):
    tf.random.set_seed(0)
    Init = keras.initializers.LecunNormal(seed=0)
    m = keras.models.Sequential([
        Flatten(input_shape=[input_shape]),
        Dense(512, activation="selu", kernel_initializer=Init),
        keras.layers.BatchNormalization(),
        Dense(256, activation="selu", kernel_initializer=Init),
        keras.layers.BatchNormalization(),
        Dense(128, activation="selu", kernel_initializer=Init),
        keras.layers.BatchNormalization(),
        Dense(64, activation="selu", kernel_initializer=Init),
        keras.layers.BatchNormalization(),
        Dense(32, activation="selu", kernel_initializer=Init),
        keras.layers.BatchNormalization(),
        keras.layers.Dropout(0.2),
        Dense(1, kernel_initializer=Init)])

    m.summary()
    m.compile(loss=tf.keras.losses.MeanSquaredError(),
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001, use_ema=True),
              metrics=[tf.keras.metrics.MeanAbsoluteError()])
    return m

In [33]:
model = DNN(input_shape=NN_input_shape*55)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 660)               0         
                                                                 
 dense (Dense)               (None, 512)               338432    
                                                                 
 batch_normalization (Batch  (None, 512)               2048      
 Normalization)                                                  
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 batch_normalization_1 (Bat  (None, 256)               1024      
 chNormalization)                                                
                                                                 
 dense_2 (Dense)             (None, 128)               3

In [34]:
%%time
# Train model
hist = model.fit(XY_dataset,
                 batch_size=512,
                 epochs=5,
                 )

Epoch 1/5
500/500 [==============================] - 22s 30ms/step - loss: 61.5344 - mean_absolute_error: 5.9736
Epoch 2/5
500/500 [==============================] - 16s 32ms/step - loss: 61.5330 - mean_absolute_error: 5.9735
Epoch 3/5
500/500 [==============================] - 16s 31ms/step - loss: 61.5318 - mean_absolute_error: 5.9735
Epoch 4/5
500/500 [==============================] - 15s 30ms/step - loss: 61.5308 - mean_absolute_error: 5.9735
Epoch 5/5
500/500 [==============================] - 15s 31ms/step - loss: 61.5298 - mean_absolute_error: 5.9734
CPU times: user 2min 57s, sys: 5.18 s, total: 3min 3s
Wall time: 2min 42s


In [35]:
sequence_length = 55
batch_size = 512

XYtest_dataset = None
for XY_test_df in tqdm(XY_test_df_lst):
    # Preprocess XY DataFrame
    Xtest_array = pp_pipeline.fit_transform(XY_test_df.drop('target', axis=1))
    # Create TF Dataset
    XYtest_ds = tf.keras.utils.timeseries_dataset_from_array(
        tf.cast(Xtest_array, tf.float32),
        XY_test_df.target,
        sequence_length,
        sequence_stride=1,
        sampling_rate=1,
        batch_size=128,
        shuffle=False,
        seed=RANDOM_STATE,
        start_index=None,
        end_index=None
    )
    if XYtest_dataset is None:
        XYtest_dataset = XYtest_ds
    else:
        XYtest_dataset = XYtest_dataset.concatenate(XYtest_ds)
    #break

100%|██████████| 50/50 [00:02<00:00, 16.69it/s]


In [36]:
model.evaluate(XYtest_dataset)

50/50 [==============================] - 3s 24ms/step - loss: 69234189139968.0000 - mean_absolute_error: 7098423.5000


[69234189139968.0, 7098423.5]

In [37]:
def get_tf_dataset(target_df, sequence_length=1, sequence_stride=1, batch_size=512):
    # Create TF Dataset
    if 'target' not in target_df:
        X_array_pp = pp_pipeline.fit_transform(target_df)
        print(f"Label 'target' was not found in target_df.\ntarget_df.columns:\n{target_df.columns}")
        Y_array = None
    else:
        X_array_pp = pp_pipeline.fit_transform(target_df.drop('target', axis=1))
        Y_array = target_df.target
    print(f"X_array_pp.shape: {X_array_pp.shape}")
    XY_ds = tf.keras.utils.timeseries_dataset_from_array(
        tf.cast(X_array_pp, tf.float32),
        Y_array,
        sequence_length,
        sequence_stride=sequence_stride,
        sampling_rate=1,
        batch_size=128,
        shuffle=False,
        seed=RANDOM_STATE,
        start_index=None,
        end_index=None
    )
    return XY_ds

In [38]:
# for index, row in XY_test_df_lst[0][:1].iterrows():
#     print(f"index:{index}")
#     display(row.to_frame().T)

In [39]:
# if not is_google_colab():
import optiver2023
env = optiver2023.make_env()
iter_test = env.iter_test()

In [40]:
from sklearn.metrics import mean_absolute_error
import pandas as pd

if is_google_colab():

    # Lists to store true and predicted targets
    true_targets = []
    predicted_targets = []

    for (test, revealed_targets, sample_prediction) in iter_test:
        # Predict target values
        print(f"test.shape: {test.shape}")
        print(f"test.columns: {test.columns}")
        predictions = None
        for _, test_item in tqdm(test.iterrows()):
            test_dataset = get_tf_dataset(test_item.to_frame().T)
            test_pred = model.predict(test_dataset)
            predictions = np.concat((predictions, test_pred)) if predictions else test_pred
        print(f"predictions.shape:{predictions.shape}")

        # Only take as many predictions and true targets as the minimum of the two
        min_len = min(len(predictions), len(revealed_targets))

        # Trim down the predictions list so that it is the same size as the revealed targets
        current_predictions = predictions[:min_len]
        current_true_targets = revealed_targets['revealed_target'].values[:min_len]

        # Extend the lists of true targets and predictions with the ones from the current batch
        true_targets.extend(current_true_targets)
        predicted_targets.extend(current_predictions)

        # Make the actual prediction and add to sample_prediction df
        sample_prediction['target'] = predictions

        # Submit the predictions for this chunk of test data
        env.predict(sample_prediction)

    # Convert lists to dataframes
    df_true = pd.DataFrame(true_targets, columns=['true_target'])
    df_pred = pd.DataFrame(predicted_targets, columns=['predicted_target'])

    # Calculate and print the MAE
    mae = mean_absolute_error(df_true.fillna(0), df_pred)
    print("Mean Absolute Error:", mae)
    # 5.54058725654896

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.
test.shape: (200, 15)
test.columns: Index(['stock_id', 'date_id', 'seconds_in_bucket', 'imbalance_size',
       'imbalance_buy_sell_flag', 'reference_price', 'matched_size',
       'far_price', 'near_price', 'bid_price', 'bid_size', 'ask_price',
       'ask_size', 'wap', 'row_id'],
      dtype='object')


0it [00:00, ?it/s]

Label 'target' was not found in target_df.
target_df.columns:
Index(['stock_id', 'date_id', 'seconds_in_bucket', 'imbalance_size',
       'imbalance_buy_sell_flag', 'reference_price', 'matched_size',
       'far_price', 'near_price', 'bid_price', 'bid_size', 'ask_price',
       'ask_size', 'wap', 'row_id'],
      dtype='object')
X_array_pp.shape: (1, 12)


ValueError: ignored

In [ ]:
if not is_google_colab():
  counter = 0
  for (test, revealed_targets, sample_prediction) in iter_test:
      print(revealed_targets.head())
      sample_prediction['target'] = model.predict(pipeline.fit_transform(test))
      env.predict(sample_prediction)
      counter += 1

In [ ]:
tmr.ShowTime()    # measure Colab's runtime.